In [209]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy
from scipy.stats import zscore

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/adidas-sales-dataset/Adidas US Sales Datasets.xlsx


In [210]:
data=pd.read_excel('/kaggle/input/adidas-sales-dataset/Adidas US Sales Datasets.xlsx')
df=data.copy()

In [211]:
# df.head()

In [228]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9649 entries, 3 to 9651
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   nan               0 non-null      float64
 1   Retailer          9649 non-null   object 
 2   Retailer ID       9649 non-null   object 
 3   Invoice Date      9649 non-null   object 
 4   Region            9649 non-null   object 
 5   State             9649 non-null   object 
 6   City              9649 non-null   object 
 7   Product           9649 non-null   object 
 8   Price per Unit    9649 non-null   object 
 9   Units Sold        9649 non-null   object 
 10  Total Sales       9649 non-null   object 
 11  Operating Profit  9649 non-null   object 
 12  Operating Margin  9649 non-null   object 
 13  Sales Method      9649 non-null   object 
dtypes: float64(1), object(13)
memory usage: 1.1+ MB


In [229]:
df = df.reset_index(drop=True)

In [214]:
df = df.drop(labels=[0,1,2], axis=0)

In [215]:
df.columns = df.iloc[0]

In [216]:
df.describe(include='all')

3,NaN,Retailer,Retailer ID,Invoice Date,Region,State,City,Product,Price per Unit,Units Sold,Total Sales,Operating Profit,Operating Margin,Sales Method
count,0.0,9649,9649.0,9649,9649,9649,9649,9649,9649.0,9649.0,9649.0,9649.0,9649.00,9649
unique,NaN,7,5.0,725,6,51,53,7,147.0,362.0,3513.0,6511.0,128.00,4
top,NaN,Foot Locker,1185732.0,2021-01-17 00:00:00,West,Texas,Portland,Men's Street Footwear,50.0,225.0,100000.0,21000.0,0.35,Online
freq,NaN,2637,5265.0,77,2448,432,360,1610,557.0,207.0,52.0,34.0,1229.00,4889
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Data cleaning: This step involves checking for missing or invalid values in the data and fixing any errors or inconsistencies.

In [230]:
def handle_missing_values(df):
    # Check for missing values in each column
    missing_values = df.isnull().sum()
    
    # Fill missing values according to data type
    for column, num_missing in missing_values.items():
        if num_missing > 0:
            if df[column].dtype == 'object':
                # Fill missing values in categorical columns with the most frequent value
                df[column].fillna(df[column].mode()[0], inplace=True)
            else:
                # Fill missing values in numerical columns with the median value
                df[column].fillna(df[column].median(), inplace=True)
                
    return df
new_df=handle_missing_values(df)

/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [231]:
new_df

3,NaN,Retailer,Retailer ID,Invoice Date,Region,State,City,Product,Price per Unit,Units Sold,Total Sales,Operating Profit,Operating Margin,Sales Method
0,NaN,Retailer,Retailer ID,Invoice Date,Region,State,City,Product,Price per Unit,Units Sold,Total Sales,Operating Profit,Operating Margin,Sales Method
1,NaN,Foot Locker,1185732,2020-01-01 00:00:00,Northeast,New York,New York,Men's Street Footwear,50,1200,600000,300000,0.5,In-store
2,NaN,Foot Locker,1185732,2020-01-02 00:00:00,Northeast,New York,New York,Men's Athletic Footwear,50,1000,500000,150000,0.3,In-store
3,NaN,Foot Locker,1185732,2020-01-03 00:00:00,Northeast,New York,New York,Women's Street Footwear,40,1000,400000,140000,0.35,In-store
4,NaN,Foot Locker,1185732,2020-01-04 00:00:00,Northeast,New York,New York,Women's Athletic Footwear,45,850,382500,133875,0.35,In-store
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9644,NaN,Foot Locker,1185732,2021-01-24 00:00:00,Northeast,New Hampshire,Manchester,Men's Apparel,50,64,3200,896.0,0.28,Outlet
9645,NaN,Foot Locker,1185732,2021-01-24 00:00:00,Northeast,New Hampshire,Manchester,Women's Apparel,41,105,4305,1377.6,0.32,Outlet
9646,NaN,Foot Locker,1185732,2021-02-22 00:00:00,Northeast,New Hampshire,Manchester,Men's Street Footwear,41,184,7544,2791.28,0.37,Outlet
9647,NaN,Foot Locker,1185732,2021-02-22 00:00:00,Northeast,New Hampshire,Manchester,Men's Athletic Footwear,42,70,2940,1234.8,0.42,Outlet
